In [1]:
# Initial imports
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import time
#import plotly.express as px

from collections import Counter
#from imblearn.combine import SMOTEENN

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# Create DataFrame from CSV
t0 = time.time()
filtered_lyric_TF_df = pd.read_csv('../Data/filtered_lyric_TF.csv')
filtered_lyric_TF_df = filtered_lyric_TF_df.drop([0,1])
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df.head(3)

Run time: 51.942492723464966 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [4]:
# Create a genre lists column
t0 = time.time()
genre_lists = []
for index, row in filtered_lyric_TF_df.iterrows():
    genres = row['genre_list'].replace('[', '').replace(']', '').replace("'", '').split(', ')
    genre_lists.append(genres)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genre_lists)

Run time: 42.26460027694702 seconds


8014

In [5]:
# Replace the genre_list column
filtered_lyric_TF_df['genre_list'] = genre_lists

In [6]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_lyric_TF_df.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 43.40717887878418 seconds
3975


4039

In [7]:
# Create non destructive copy and remove the unnecessary indeces
t0 = time.time()
filtered_lyric_TF_df2 = filtered_lyric_TF_df.drop(indeces_to_remove)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(filtered_lyric_TF_df2))
filtered_lyric_TF_df2.head()

Run time: 0.3473787307739258 seconds
3975


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BOYSHIT,Madison Beer,pop,8.0,"[dance, pop, electropop, post-teen]",0.638,0.699,1.0,1.0,0.0892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Safe With Me (with Audrey Mika),Gryffin,pop,8.0,"[dance, pop, tropical, edm]",0.497,0.757,11.0,1.0,0.1270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Remove the romance and punk categories
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='romance']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='punk']
len(filtered_lyric_TF_df2)

3759

In [9]:
# Define the features set
X = filtered_lyric_TF_df2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
X.head()

,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,0.392,0.574,7.0,1.0,0.1700,0.8330,0.00179,0.1450,0.529,81.112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.581,0.584,2.0,1.0,0.2840,0.0805,0.00000,0.3660,0.756,179.954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.666,0.593,2.0,1.0,0.0373,0.3180,0.00000,0.4140,0.329,140.013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.638,0.699,1.0,1.0,0.0892,0.0203,0.00000,0.0879,0.570,121.122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.497,0.757,11.0,1.0,0.1270,0.0449,0.00000,0.1190,0.507,169.902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Define the target set
y = filtered_lyric_TF_df2['category_id'].ravel()
y[:5]

array([8., 8., 8., 8., 8.])

In [11]:
# Standardize data with StandardScaler
X_scaled = StandardScaler().fit_transform(X)

In [12]:
# Create a dictionary with n_components and explained variance sums
pca_models = {}
n_components = [2, 50, 500, 1000]
for n in n_components:
    # Initialize PCA model
    pca = PCA(n_components=n)
    
    # Get principal components for the data
    t0 = time.time()
    lyric_pca = pca.fit_transform(X)
    t1 = time.time()
    print(f'Run time for {n} components: {t1-t0} seconds')
    
    # Transform PCA data to a DataFrame
    columns = []
    for i in range(len(lyric_pca[0])):
        columns.append(f'PC {i+1}')
    lyric_pca_df = pd.DataFrame(
        data=lyric_pca, columns=columns)
    
    # Get explained variance sum
    pca_models[n] = sum(pca.explained_variance_ratio_)
    
    # Initialize and fit the K-means model
    model = KMeans(n_clusters=12, random_state=0)
    model.fit(lyric_pca_df)
    
    # Add a new predictions to filtered_lyric_TF_df2
    predictions = model.predict(lyric_pca_df)
    column_name = 'k-means' + str(n)
    filtered_lyric_TF_df2[column_name] = predictions
print(pca_models)
filtered_lyric_TF_df2.head(3)

Run time for 2 components: 2.3404347896575928 seconds
Run time for 50 components: 4.243023157119751 seconds
Run time for 500 components: 10.947726011276245 seconds
Run time for 1000 components: 17.76934504508972 seconds
{2: 0.9999997457629796, 50: 0.9999998667815421, 500: 0.9999999672765616, 1000: 0.9999999872014311}


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,nothingness,amused,corners,interlude,sting,axis,k-means2,k-means50,k-means500,k-means1000
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,5,5,5,7
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,5,5,5,7
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2,1


In [13]:
# Create tuples of categories and k-means predictions for each n_components
t0 = time.time()
category_predictions2 = []
category_predictions50 = []
category_predictions500 = []
category_predictions1000 = []
for index, row in filtered_lyric_TF_df2.iterrows():
    category_predictions2.append((row['category_id'], row['k-means2']))
    category_predictions50.append((row['category_id'], row['k-means50']))
    category_predictions500.append((row['category_id'], row['k-means500']))
    category_predictions1000.append((row['category_id'], row['k-means1000']))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(category_predictions2[0:5])
print(category_predictions50[0:5])
print(category_predictions500[0:5])
print(category_predictions1000[0:5])

Run time: 20.778312921524048 seconds
[(8.0, 5), (8.0, 5), (8.0, 2), (8.0, 6), (8.0, 5)]
[(8.0, 5), (8.0, 5), (8.0, 2), (8.0, 6), (8.0, 5)]
[(8.0, 5), (8.0, 5), (8.0, 2), (8.0, 6), (8.0, 5)]
[(8.0, 7), (8.0, 7), (8.0, 1), (8.0, 9), (8.0, 7)]


In [49]:
# Define a function that returns a list of unique combinations
def filter_list(predictions, items_to_remove):
    preds = predictions.copy()
    for i in preds:
        if i[0] in items_to_remove:
            preds.remove(i)
    return preds

def combinations(combo, preds):
    return ([old, new]
               for old in combo
               for new in preds )

def get_unique_combos(category_preds, category_list, unique_pred, mins):
    predictions = category_preds[0.0]
    combos_of_two = list(combinations(combo=unique_pred, preds=predictions))
    
    prev_combos = combos_of_two
    for index, cat in enumerate(category_list[0:6]):
        t0 = time.time()
        new_combos = []
        for combo in prev_combos:
            predictions = filter_list(category_preds[cat], [i[0] for i in combo[1:]])
            new_preds = list(combinations(combo=unique_pred, preds=predictions))
            new_combos.extend([i+combo[1:] for i in new_preds])
        #prev_combos = [combo for combo in new_combos if len(set([i[0] for i in combo]))==index+3]
        prev_combos = new_combos
        t1 = time.time()
        #print(f'Run time to find {len(new_combos)} combos of {index+3}: {t1-t0} seconds')
    
    for index, cat in enumerate(category_list[6:]):
        t0 = time.time()
        prev_combos = [combo for combo in prev_combos if sum([i[1] for i in combo]) > mins[index]]
        new_combos = []
        for combo in prev_combos:
            predictions = filter_list(category_preds[cat], [i[0] for i in combo[1:]])
            new_preds = list(combinations(combo=unique_pred, preds=predictions))
            new_combos.extend([i+combo[1:] for i in new_preds])
        prev_combos = new_combos
        t1 = time.time()
        #print(f'Run time to find {len(new_combos)} combos of {index+9}: {t1-t0} seconds')
    unique_combos = [combo for combo in new_combos if len(set([i[0] for i in combo]))==12]
    return(unique_combos)


In [50]:
# Find the most popular predictions for each category when n_components=2
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in category_predictions2:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        prediction_counts.append((prediction, predictions.count(prediction)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    #print(category, category_preds[category])

# Find the optimal combination
category_list = list(set(filtered_lyric_TF_df2['category_id']))
category_list.remove(1.0)
category_list.remove(0.0)
unique_pred = [(9, 2)]

t0 = time.time()
unique_combos = get_unique_combos(category_preds=category_preds, category_list=category_list, 
                                  unique_pred=unique_pred, mins = [439, 530, 585, 641])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'{len(unique_combos)} combinations found')

most = 0
for combo in unique_combos:
    total = sum([i[1] for i in combo])
    if total > most:
        most = total
        best_combo = combo
print(best_combo)

total = 3934
print(f'{most} correct predictions out of {total}')
print(f'Accuracy: {most/total}')

Run time: 72.45096325874329 seconds
4974 combinations found
[(9, 2), (6, 59), (10, 10), (8, 51), (5, 87), (3, 66), (4, 3), (0, 74), (11, 54), (1, 14), (2, 286), (7, 21)]
727 correct predictions out of 3934
Accuracy: 0.18479918657854602


In [127]:
# Find the most popular predictions for each category when n_components=50
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in category_predictions50:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        prediction_counts.append((prediction, predictions.count(prediction)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    print(category, category_preds[category])

# Check for any unique predictions
preds_df = pd.DataFrame(columns=list(range(0, 12)))
preds_df.loc[0] = list(set(filtered_lyric_TF_df2['category_id']))
preds_df.loc[1] = 0
preds_df.loc[2] = 0
for cat in category_preds:
    for i in category_preds[cat]:
        preds_df.loc[1, i[0]]+=1
        preds_df.loc[2, i[0]] = cat
        preds_df
#display(preds_df)

# Find the optimal combination
category_list = list(set(filtered_lyric_TF_df2['category_id']))
category_list.remove(1.0)
category_list.remove(0.0)
unique_pred = [(9, 1)]

t0 = time.time()
unique_combos = get_unique_combos(category_preds=category_preds, category_list=category_list, 
                                  unique_pred=unique_pred, mins = [439, 530, 585, 641])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'{len(unique_combos)} combinations found')

most = 0
for combo in unique_combos:
    total = sum([i[1] for i in combo])
    if total > most:
        most = total
        best_combo = combo
print(best_combo)

total = 3934
print(f'{most} correct predictions out of {total}')
print(f'Accuracy: {most/total}')

0.0 [(2, 63), (6, 50), (5, 45), (0, 40), (8, 32), (3, 28), (7, 22), (11, 17), (1, 10), (10, 8)]
1.0 [(8, 9), (0, 6), (4, 4), (1, 3), (5, 2), (7, 2), (2, 1), (3, 1), (6, 1), (9, 1), (11, 1)]
2.0 [(2, 284), (5, 251), (6, 171), (0, 128), (8, 68), (11, 37), (3, 31), (7, 6), (1, 1)]
3.0 [(3, 25), (5, 22), (8, 22), (0, 21), (7, 20), (1, 14), (2, 13), (6, 10), (10, 9), (11, 6), (4, 1)]
4.0 [(6, 81), (5, 69), (0, 62), (2, 59), (11, 54), (3, 47), (8, 46), (7, 24), (1, 3), (10, 1)]
5.0 [(5, 78), (0, 74), (2, 72), (8, 41), (3, 37), (6, 32), (7, 18), (11, 11), (1, 9), (10, 3)]
6.0 [(3, 13), (0, 9), (2, 9), (5, 9), (8, 9), (7, 8), (6, 7), (1, 6), (4, 3), (10, 2), (11, 2)]
7.0 [(8, 72), (3, 68), (0, 56), (7, 41), (5, 36), (1, 26), (2, 25), (6, 18), (10, 16), (11, 5), (4, 3)]
8.0 [(2, 91), (5, 83), (6, 81), (0, 53), (8, 17), (11, 10), (3, 6), (7, 2)]
10.0 [(8, 55), (0, 47), (5, 34), (3, 27), (2, 25), (6, 11), (7, 8), (1, 3), (11, 3)]
11.0 [(8, 56), (3, 35), (5, 31), (6, 26), (2, 24), (0, 22), (7, 16)

In [128]:
# Find the most popular predictions for each category when n_components=500
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in category_predictions500:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        prediction_counts.append((prediction, predictions.count(prediction)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    #print(category, category_preds[category])

# Find the optimal combination
category_list = list(set(filtered_lyric_TF_df2['category_id']))
category_list.remove(1.0)
category_list.remove(0.0)
unique_pred = [(9, 1)]

t0 = time.time()
unique_combos = get_unique_combos(category_preds=category_preds, category_list=category_list, 
                                  unique_pred=unique_pred, mins = [439, 530, 585, 641])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'{len(unique_combos)} combinations found')

most = 0
for combo in unique_combos:
    total = sum([i[1] for i in combo])
    if total > most:
        most = total
        best_combo = combo
print(best_combo)

total = 3934
print(f'{most} correct predictions out of {total}')
print(f'Accuracy: {most/total}')

Run time: 65.24452185630798 seconds
2926 combinations found
[(9, 1), (6, 59), (10, 10), (8, 55), (5, 79), (3, 71), (4, 3), (0, 72), (11, 54), (1, 14), (2, 281), (7, 22)]
721 correct predictions out of 3934
Accuracy: 0.18327402135231316


In [130]:
# Find the most popular predictions for each category when n_components=1000
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in category_predictions1000:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        prediction_counts.append((prediction, predictions.count(prediction)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    #print(category, category_preds[category])

# Find the optimal combination
category_list = list(set(filtered_lyric_TF_df2['category_id']))
category_list.remove(1.0)
category_list.remove(0.0)
unique_pred = [(5, 1)]

t0 = time.time()
unique_combos = get_unique_combos(category_preds=category_preds, category_list=category_list, 
                                  unique_pred=unique_pred, mins = [435, 529, 584, 642])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'{len(unique_combos)} combinations found')

most = 0
for combo in unique_combos:
    total = sum([i[1] for i in combo])
    if total > most:
        most = total
        best_combo = combo
print(best_combo)

total = 3934
print(f'{most} correct predictions out of {total}')
print(f'Accuracy: {most/total}')

Run time: 61.41184997558594 seconds
1399 combinations found
[(5, 1), (9, 55), (4, 10), (8, 55), (1, 94), (0, 73), (10, 3), (3, 73), (11, 52), (2, 14), (7, 260), (6, 22)]
712 correct predictions out of 3934
Accuracy: 0.1809862735129639


### 20, 30, 40, and 100 components also result in 727 correct predictions

In [99]:
# Initialize and fit the K-means model
model = KMeans(n_clusters=12, random_state=0)
model.fit(lyric_pca_df)

# Predict clusters
#predictions = model.predict(lyric_pca_df)

# Add the predicted class columns
lyric_pca_df['class'] = model.labels_
display(lyric_pca_df.head())


,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,PC 7,PC 8,PC 9,PC 10,...,PC 492,PC 493,PC 494,PC 495,PC 496,PC 497,PC 498,PC 499,PC 500,class
0,-0.597476,-0.223994,-0.664207,-0.453666,1.010867,-0.151724,0.883912,1.085227,-0.194524,0.330865,...,-1.555025,5.711150,2.046121,-0.054366,-2.209421,3.969913,2.862224,-10.324893,-4.591399,0
1,0.323348,-0.555811,-0.216781,0.241110,0.071625,-0.424917,0.117996,0.582056,-0.365542,-0.212150,...,-1.739095,1.148711,0.437583,0.656007,1.652691,1.329667,-0.701451,-0.847759,0.882228,0
2,-0.563524,-0.634686,-1.280968,0.017776,0.049061,0.048332,0.168502,-0.300357,-1.228525,-0.609979,...,-0.665176,0.670216,-0.570384,-1.777787,-0.135929,-0.936574,0.272468,1.976609,-2.736178,0
3,-0.592278,-0.258136,1.266419,2.008088,2.044069,-0.085380,0.114914,0.325381,-1.038714,-0.800322,...,0.107794,-0.229452,1.625582,-0.977467,-0.262011,-0.687067,-0.255469,1.161737,-0.497776,0
4,-1.242838,-0.549866,-1.607882,0.181408,0.316094,-0.011521,0.256745,0.383781,-0.378493,-0.298848,...,0.073817,0.151581,-0.987037,-0.917522,0.379553,0.571921,0.674525,0.672091,1.643627,0


In [102]:
# Add a new k-means column to filtered_lyric_TF_df2
filtered_lyric_TF_df2['k-means'] = model.labels_
filtered_lyric_TF_df2.head()

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,amused,corners,interlude,sting,axis,k-means,k-means2,k-means50,k-means500,k-means1000
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0,6,0,3,1
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0,6,0,3,1
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0,6,0,3,1
5,BOYSHIT,Madison Beer,pop,8.0,"[dance, pop, electropop, post-teen]",0.638,0.699,1.0,1.0,0.0892,...,0.0,0.0,0.0,0.0,0.0,0,6,0,3,1
6,Safe With Me (with Audrey Mika),Gryffin,pop,8.0,"[dance, pop, tropical, edm]",0.497,0.757,11.0,1.0,0.1270,...,0.0,0.0,0.0,0.0,0.0,0,0,0,3,1


In [103]:
# Create a tuple of categories and k-means predictions
t0 = time.time()
category_predictions = []
for index, row in filtered_lyric_TF_df2.iterrows():
    category_predictions.append((row['category_id'], row['k-means']))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
category_predictions

Run time: 25.191790103912354 seconds


[(8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),
 (8.0, 0),

In [104]:
# Find the most popular predictions for each category
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in category_predictions:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        prediction_counts.append((prediction, predictions.count(prediction)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    print(category, category_preds[category])

0.0 [(0, 315)]
1.0 [(0, 31)]
2.0 [(0, 977)]
3.0 [(0, 163)]
4.0 [(0, 434), (10, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1)]
5.0 [(0, 375)]
6.0 [(0, 77)]
7.0 [(0, 366)]
8.0 [(0, 343)]
10.0 [(0, 213)]
11.0 [(0, 245)]
13.0 [(0, 208)]


In [ ]:

blues 0.0     - 4  (10)  
classical 1.0 - 3  (1)  
country 2.0   - 6  (394)   
funk 3.0      - 10 (27)  
hiphop 4.0    - 11 (66)
indie_alt 5.0 - 5  (85) 
jazz 6.0      - 1  (1) 
metal 7.0     - 2  (78)  
pop 8.0       - 0  (146) 
rnb 10.0      - 9  (39) 
rock 11.0     - 7  (1)  
soul 13.0     - 8  (0)  


In [232]:
# Print the accuracy percentage
total = 184 + 716 + 534 + 54 + 616 + 373 + 1 + 12 + 486 + 697 + 95 + 166
correct = 10 + 1 + 394 + 27 + 66 + 85 + 1 + 78 + 146 + 39 + 1
print(f'{correct} correct predictions out of {total}')
print(f'Accuracy: {correct/total}')

848 correct predictions out of 3934
Accuracy: 0.215556685307575


In [ ]:
# Find the most popular predictions for each category
category_preds = {}
category_list = set(filtered_lyric_TF_df2['category_id'])
for category in category_list:
    predictions = []
    for pair in Scategory_predictions:
        if pair[0] == category:
            predictions.append(pair[1])
    unique_preds = list(set(predictions))
    prediction_counts = []
    for prediction in unique_preds:
        #print(category, prediction, predictions.count(prediction))
        prediction_counts.append((prediction, predictions.count(prediction)))
        #prediction_counts.append((prediction, predictions.count(prediction)/len(predictions)))
    category_preds[category] = sorted(prediction_counts, key = lambda x: x[1], reverse=True)#[0:3]
    print(category, category_preds[category])

In [ ]:

blues 0.0     - 4  (10)  
classical 1.0 - 3  (1)  
country 2.0   - 6  (394)   
funk 3.0      - 10 (27)  
hiphop 4.0    - 11 (66)
indie_alt 5.0 - 5  (85) 
jazz 6.0      - 1  (1) 
metal 7.0     - 2  (78)  
pop 8.0       - 0  (146) 
rnb 10.0      - 9  (39) 
rock 11.0     - 7  (1)  
soul 13.0     - 8  (0)  


In [ ]:
# Print the accuracy percentage
total = 184 + 716 + 534 + 54 + 616 + 373 + 1 + 12 + 486 + 697 + 95 + 166
correct = 10 + 1 + 394 + 27 + 66 + 85 + 1 + 78 + 146 + 39 + 1
print(f'{correct} correct predictions out of {total}')
print(f'Accuracy: {correct/total}')

In [32]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6010, 12020)
(2004, 12020)
(6010,)
(2004,)


In [24]:
# Initializing model with K=14
X = filtered_lyric_TF_df.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
model = KMeans(n_clusters=14, random_state=5)
model

KMeans(n_clusters=14, random_state=5)

In [29]:
# Fitting mode
model.fit(X, y)

KMeans(n_clusters=14, random_state=5)

In [30]:
# Get predicitions
predictions = model.predict(X, y)
print(predictions)

[0 6 2 ... 3 4 0]


In [31]:
# Get predicitions
predictions = model.predict(filtered_lyric_TF_df)
print(predictions)

ValueError: could not convert string to float: 'willow'

In [19]:
# Fitting mode
model.fit(X)

KMeans(n_clusters=14, random_state=5)

In [ ]:
# Get predicitions
predictions = model.predict(X)
print(predictions)

In [ ]:
# Initializing model with K=3 
model = KMeans(n_clusters=3, random_state=5)
model